# Alura Challenge - Semana 03

## Objetivos deste trabalho

•	Verificar se a variável target está balanceada;

•	Aplicar encoding nos seus dados;

•	Criar dois ou mais modelos de Machine Learning;

•	Avaliar cada modelo utilizando métricas de ML;

•	Escolher o melhor modelo;

•	Otimizar o melhor modelo;

•	Verificar qual o melhor tipo de balanceamento com esses dados.

## Verificando o balanceamento da variável target Churn

In [68]:
df_churn = pd.read_csv('df_churn.csv')
df_churn.head()

,ID_Cliente,Churn,Genero,Idoso,Parceiro,Dependentes,Meses_Contrato,Servico_Telefonico,Multiplas_Linhas,Internet,Seguranca_Online,Backup_online,Protecao_Dispositivo,Suporte_Tecnico,Streaming_TV,Streaming_Filmes,Fatura_Online,Gasto_diario,Gasto_Mensal,Gasto_Total,Mensal,Anual,Bianual,Transf_banco,Cartao_credito,Boleto_eletronico,Boleto_correios
0,0002-ORFBO,0,0,0,1,1,9,1,0,1,0,1,0,1,1,0,1,2.19,65.6,593.30,0,1,0,0,0,0,1
1,0003-MKNFE,0,1,0,0,0,9,1,1,1,0,0,0,0,0,1,0,2.00,59.9,542.40,1,0,0,0,0,0,1
2,0004-TLHLJ,1,1,0,0,0,4,1,0,1,0,0,1,0,0,0,1,2.46,73.9,280.85,1,0,0,0,0,1,0
3,0011-IGKFF,1,1,1,1,0,13,1,0,1,0,1,1,0,1,1,1,3.27,98.0,1237.85,1,0,0,0,0,1,0
4,0013-EXCHZ,1,0,1,1,0,3,1,0,1,0,0,0,1,1,0,1,2.80,83.9,267.40,1,0,0,0,0,0,1


In [69]:
proporcao_churn = round(dataframe_final.Churn.value_counts(normalize = True)*100, 2)
proporcao_churn

0    73.46
1    26.54
Name: Churn, dtype: float64

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Conforme anteriormente avaliado, a variável target Churn está dividida em, aproximadamente, 73,5% 'Não' e 26,5% 'Sim' dos dados do nosso dataset. Desta forma, há um desbalanceamento entre estes valores, o que poderá ocasionar enviesamento e conseguinte interferência no nosso modelo de previsão. Para mitigar este efeito, utilizaremos algumas técnicas de rebalanceamentos de dados: </p>

## Tratando o desbalanceamento da variável target Churn

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Para tratarmos o desbalanceamento da nossa variável target, duas técnicas consagradas poderão ser utilizadas: undersampling (redução da quantidade de dados do valor 'Não') e oversampling' (ampliação dos dados do valor 'Sim'). Ambas técnicas possuem vantagens e desvantagens intrínsecas. Para este trabalho, iremos avaliar as duas possibilidades de modo a comparar o resultado final obtido em cada técnica. Antes, no entanto, devemos proceder com o pré-processameto dos dados, ou seja, dividi-los entre dados de treino e dados de teste, e também normalizar as features 'Meses_Contrato', 'Gasto_total', 'Gasto_Mensal'. Este procedimento inicial é necessário para evitarmos o chamado 'data leakage' e o conseguinte enviesamento dos modelos.</p>

## Pré-processamento dos dados

In [70]:
y = df_churn['Churn']
X = df_churn[['Idoso', 'Dependentes', 'Meses_Contrato', 'Internet', 'Fatura_Online', 'Gasto_Mensal', 'Gasto_Total', 'Parceiro',
                      'Mensal', 'Bianual', 'Cartao_credito', 'Boleto_eletronico', 'Boleto_correios', 'Transf_banco', 'Anual']]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 53)

In [72]:
features_numericas = ['Meses_Contrato', 'Gasto_Total', 'Gasto_Mensal']

In [73]:
X_train_normal = X_train.copy()
X_test_normal = X_test.copy()


for i in features_numericas:
    
    scaler = MinMaxScaler()
    scaler.fit(X_train_normal[[i]])
    scaler.fit(X_test_normal[[i]])

    X_train_normal[[i]] = scaler.transform(X_train_normal[[i]])
    X_test_normal[[i]] = scaler.transform(X_test_normal[[i]])


In [74]:
X_train_normal.head()

,Idoso,Dependentes,Meses_Contrato,Internet,Fatura_Online,Gasto_Mensal,Gasto_Total,Parceiro,Mensal,Bianual,Cartao_credito,Boleto_eletronico,Boleto_correios,Transf_banco,Anual
5498,0,0,0.708333,1,1,0.117530,0.174310,1,1,0,1,0,0,0,0
2611,0,0,0.208333,0,0,0.015438,0.032968,0,0,0,0,0,1,0,1
1282,0,0,0.013889,0,0,0.012450,0.000069,0,1,0,0,0,1,0,0
5878,0,0,0.277778,1,1,0.504482,0.155950,0,1,0,1,0,0,0,0
5289,0,0,0.736111,1,1,0.426793,0.385303,0,0,0,0,1,0,0,1


In [75]:
X_test_normal.head()

,Idoso,Dependentes,Meses_Contrato,Internet,Fatura_Online,Gasto_Mensal,Gasto_Total,Parceiro,Mensal,Bianual,Cartao_credito,Boleto_eletronico,Boleto_correios,Transf_banco,Anual
6744,0,0,0.083333,0,0,0.068227,0.017552,1,0,0,0,0,1,0,1
2840,0,0,0.888889,1,1,0.804283,0.733553,1,0,1,0,0,0,1,0
6629,0,1,0.708333,1,1,0.716135,0.523425,1,0,0,0,1,0,0,1
2593,0,0,0.958333,1,0,0.805777,0.791199,1,0,1,0,0,0,1,0
6858,0,0,0.041667,1,0,0.068227,0.007028,0,1,0,0,1,0,0,0


### Undersampling

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Para a técnica de Undersampling, utilizaremos o método NearMiss, que considera a menor distância média entre K-vizinhos mais próximos.</p>

In [76]:
X_train_normal_nm = X_train_normal.copy()
X_test_normal_nm = X_test_normal.copy()
y_train_nm = y_train.copy()
y_test_nm = y_test.copy()

In [77]:
nm = NearMiss(version = 2)
X_nm_train, y_nm_train = nm.fit_resample(X_train_normal_nm, y_train_nm)

In [78]:
nm = NearMiss(version = 2)
X_nm_test, y_nm_test = nm.fit_resample(X_test_normal_nm, y_test_nm)

In [79]:
print(f'Número de dados de treino pré-balanceamento: {y_train.shape}')
print(f'Número de dados de treino pós-balanceamento: {y_nm_train.shape}')
print(f'Número de dados de teste pré-balanceamento: {y_test.shape}')
print(f'Número de dados de teste pós-balanceamento: {y_nm_test.shape}')

Número de dados de treino pré-balanceamento: (5634,)
Número de dados de treino pós-balanceamento: (3028,)
Número de dados de teste pré-balanceamento: (1409,)
Número de dados de teste pós-balanceamento: (710,)


In [80]:
nova_proporcao_churn = round(y_nm_train.value_counts(normalize = True)*100, 2)
nova_proporcao_churn

0    50.0
1    50.0
Name: Churn, dtype: float64

### Oversampling

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Para a técnica de Oversampling, utilizaremos o método SMOTE, que cria dados sintéticos para a classe de menor quantidade proporcional de dados.</p>

In [81]:
X_train_normal_sm = X_train_normal.copy()
X_test_normal_sm = X_test_normal.copy()
y_train_sm = y_train.copy()
y_test_sm = y_test.copy()

In [82]:
sm = SMOTE(random_state = 53, k_neighbors = 5)
X_sm_train, y_sm_train = sm.fit_resample(X_train_normal_sm, y_train_sm)

In [83]:
sm = SMOTE(random_state = 53, k_neighbors = 5)
X_sm_test, y_sm_test = sm.fit_resample(X_test_normal_sm, y_test_sm)

In [84]:
print(f'Número de dados de treino pré-balanceamento: {y_train.shape}')
print(f'Número de dados de treino pós-balanceamento: {y_sm_train.shape}')
print(f'Número de dados de teste pré-balanceamento: {y_test.shape}')
print(f'Número de dados de teste pós-balanceamento: {y_sm_test.shape}')

Número de dados de treino pré-balanceamento: (5634,)
Número de dados de treino pós-balanceamento: (8240,)
Número de dados de teste pré-balanceamento: (1409,)
Número de dados de teste pós-balanceamento: (2108,)


In [85]:
nova_proporcao_churn = round(y_sm_train.value_counts(normalize = True)*100, 2)
nova_proporcao_churn

0    50.0
1    50.0
Name: Churn, dtype: float64

## Criando modelos de Machine Learning para predizer o Churn

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Estamos aptos, neste momento, a criar nossos modelos preditivos baseados nas features previamente selecionadas e nas variáveis balanceadas. Optamos por utilizar os principais modelos de machine learning de classificação: Logistic Regression, Random Forest Classifier, Decision Tree Classifier, Gradient Boosting Classifier e Support Vector Classifier. Antes, porém, iremos iniciar com o modelo comparador DummyClassifier baseado no dataframe pré-balanceamento. Obs.: Nossas features foram previamente classificadas numericamente em 0 e 1 (para as variáveis originalmente categóricas do nosso dataset). Desta forma, não será necessário utilizarmos Enconding no dataframe.</p>

In [86]:
modelos = {'Logistic Regression': LogisticRegression(random_state = 53, max_iter = 200, solver = 'lbfgs'), 
           'Random Forest Classifier': RandomForestClassifier(random_state = 53, max_depth = 15, n_estimators = 100),
           'Decision Tree Classifier': DecisionTreeClassifier(random_state = 53, max_depth = 6, criterion = 'gini'),
           'Gradient Boosting Classifier': GradientBoostingClassifier(n_estimators = 100, max_depth = 3, min_samples_split = 2,
                                                              learning_rate = 0.1, random_state = 53),
           'Support Vector Classifier': svm.SVC(kernel = 'rbf', random_state = 53)}

### DummyClassifier

In [87]:
modelo_dummy = DummyClassifier(strategy = 'stratified', random_state = 53)

In [88]:
modelo_dummy = modelo_dummy.fit(X_train_normal, y_train)

In [89]:
y_pred_dummy = modelo_dummy.predict(X_test_normal)

In [90]:
y_test_dummy = y_test

### Undersampling

In [91]:
y_pred_nm = {}
y_teste_nm ={}

for nome, modelo in modelos.items():
    modelo.fit(X_nm_train, y_nm_train)
    y_pred_nm[nome] = modelo.predict(X_nm_test)
    y_teste_nm[nome] = y_nm_test

### Oversampling

In [92]:
y_pred_sm = {}
y_teste_sm ={}

for nome, modelo in modelos.items():
    modelo.fit(X_sm_train, y_sm_train)
    y_pred_sm[nome] = modelo.predict(X_sm_test)
    y_teste_sm[nome] = y_sm_test

## Avaliando os modelos criados

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Uma vez que nossos modelos estejam instanciados e treinados, podemos partir para a avaliação dos mesmos. Para tanto, nos valeremos do método classification_report da biblioteca sklearn, a qual disponibiliza algumas métricas importantes como a Precisão (do total de usuários que o modelo previu como positivos, quantos realmente eram positivos), Revocação (do total de usuários que realmente eram positivos, quantos o modelo previu como positivos) e f1-score (média harmônica enter a Precisão e a Revocação). Primeiramente, iremos avaliar nosso modelo comparador DummyClassifier pré-balanceamento:</p>

### DummyClassifier

In [93]:
DummyClassifier = classification_report(y_test_dummy, y_pred_dummy)
print(DummyClassifier)

              precision    recall  f1-score   support

           0       0.75      0.76      0.75      1054
           1       0.25      0.25      0.25       355

    accuracy                           0.63      1409
   macro avg       0.50      0.50      0.50      1409
weighted avg       0.62      0.63      0.63      1409



<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Aqui, podemos perceber claramente a necessidade de balanceamento dos dados: a acurácia do modelo Dummy Classifier foi de 0.63, porém analisando especificamente as métricas relativas ao Churn 'Sim' (ou 1), verificamos que há um claro enviesamento do modelo em favor do valor 'Não', indo ao encontro da proporção anteriormente calculada.</p>

### Undersampling

In [94]:
for nome, y_pred in y_pred_nm.items():
    print(f'{nome}:\n')
    for nome_nm, y_teste in y_teste_nm.items():
        if nome == nome_nm:
            print(classification_report(y_teste, y_pred))
    print('\n')

Logistic Regression:

              precision    recall  f1-score   support

           0       0.75      0.76      0.75       355
           1       0.75      0.74      0.75       355

    accuracy                           0.75       710
   macro avg       0.75      0.75      0.75       710
weighted avg       0.75      0.75      0.75       710



Random Forest Classifier:

              precision    recall  f1-score   support

           0       0.74      0.78      0.76       355
           1       0.77      0.73      0.75       355

    accuracy                           0.75       710
   macro avg       0.76      0.75      0.75       710
weighted avg       0.76      0.75      0.75       710



Decision Tree Classifier:

              precision    recall  f1-score   support

           0       0.73      0.81      0.77       355
           1       0.79      0.70      0.74       355

    accuracy                           0.76       710
   macro avg       0.76      0.76      0.76     

### Oversampling

In [95]:
for nome, y_pred in y_pred_sm.items():
    print(f'{nome}:\n')
    for nome_sm, y_teste in y_teste_sm.items():
        if nome == nome_sm:
            print(classification_report(y_teste, y_pred))
    print('\n')

Logistic Regression:

              precision    recall  f1-score   support

           0       0.82      0.74      0.78      1054
           1       0.77      0.84      0.80      1054

    accuracy                           0.79      2108
   macro avg       0.79      0.79      0.79      2108
weighted avg       0.79      0.79      0.79      2108



Random Forest Classifier:

              precision    recall  f1-score   support

           0       0.74      0.78      0.76      1054
           1       0.77      0.73      0.75      1054

    accuracy                           0.76      2108
   macro avg       0.76      0.76      0.76      2108
weighted avg       0.76      0.76      0.76      2108



Decision Tree Classifier:

              precision    recall  f1-score   support

           0       0.78      0.75      0.77      1054
           1       0.76      0.79      0.78      1054

    accuracy                           0.77      2108
   macro avg       0.77      0.77      0.77     

## Escolhendo o melhor modelo e o tipo de balanceamento mais adequado

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Para nosso problema principal de redução da taxa de churn dos clientes da Alura Voz, queremos otimizar a identificação dos clientes com propensão a evadir do serviço, de modo a tentarmos estratégias comerciais de retenção dos mesmos na base. Sendo assim, diante dos resultados apresentados anteriormente, podemos perceber que o modelo Gradient Boosting Classifier, juntamente com a técnica oversampling, apresentou o melhor score integrado, seja de precisão, seja de revocação. Portanto, este será o modelo escolhido para o nosso dataset.</p>

## Otimizando o melhor modelo

<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Podemos, ainda, tentar otimizar os hiperparâmetros do nosso modelo escolhido e avaliar a sua performance com os dados balanceados. Para isto, utilizaremos a biblioteca GridSearchCV, que realiza a validação cruzada para cada variação dos parâmetros escolhidos.</p>

In [96]:
modelo_escolhido = GradientBoostingClassifier()

In [97]:
hiperparametros = {'n_estimators': [100, 300, 500, 600],
                   'max_depth': [3, 4, 5, 6, 7],
                   'min_samples_split': [2, 3, 4, 5],
                   'learning_rate': [0.1, 1]}

In [98]:
otimiz = GridSearchCV(modelo_escolhido, hiperparametros, cv = 5,  verbose = 3, n_jobs = 1)
otimiz.fit(X_sm_train, y_sm_train)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV 1/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=100;, score=0.768 total time=   1.0s
[CV 2/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=100;, score=0.773 total time=   1.0s
[CV 3/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=100;, score=0.777 total time=   1.0s
[CV 4/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=100;, score=0.792 total time=   1.0s
[CV 5/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=100;, score=0.785 total time=   1.0s
[CV 1/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=300;, score=0.747 total time=   3.0s
[CV 2/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=300;, score=0.769 total time=   3.0s
[CV 3/5] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=300;, score=0.791 total time=   2.9s
[CV 4/5] 

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=1,
             param_grid={'learning_rate': [0.1, 1],
                         'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 300, 500, 600]},
             verbose=3)

In [99]:
best_par = otimiz.best_params_
print(best_par)

{'learning_rate': 0.1, 'max_depth': 7, 'min_samples_split': 5, 'n_estimators': 300}


In [100]:
modelo_churn_otimizado = GradientBoostingClassifier(max_depth = best_par['max_depth'],
                                                    min_samples_split = best_par['min_samples_split'], 
                                                    n_estimators = best_par['n_estimators'],
                                                    learning_rate = best_par['learning_rate'], random_state = 53)

In [101]:
modelo_churn_otimizado = modelo_churn_otimizado.fit(X_sm_train, y_sm_train)

In [102]:
y_pred_modelo_churn_otimizado = modelo_churn_otimizado.predict(X_sm_test)

In [103]:
y_test_modelo_churn_otimizado = y_sm_test

In [104]:
print(classification_report(y_test_modelo_churn_otimizado, y_pred_modelo_churn_otimizado))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83      1054
           1       0.83      0.81      0.82      1054

    accuracy                           0.82      2108
   macro avg       0.82      0.82      0.82      2108
weighted avg       0.82      0.82      0.82      2108



<p style='font-size: 16px; line-height: 2; margin: 10px 50px; text-align: justify;'>Percebemos que houve uma melhora discreta na qualidade de predição do nosso modelo escohido. Por fim, iremos calcular o intervalo (95% de confiança) relativo ao score do modelo para os melhores parâmetros selecionados e salvá-lo para posterior deployment.</p>

In [105]:
media = otimiz.best_score_
desvio = otimiz.cv_results_['std_test_score'][otimiz.best_index_]
print("Intervalo de confiança [%.2f, %.2f]" % (media - 2 * desvio, media + 2 * desvio))

Intervalo de confiança [0.71, 0.90]


In [106]:
modelo_final = 'modelo_churn.sav'
joblib.dump(modelo_churn_otimizado, modelo_final)